In [1]:
import pickle
import pandas as pd
import requests
import time
import json
from glob import glob

import seaborn as sns

In [2]:
dskey = pickle.load(open("darkskyapi.p",'rb'))
open("darkskyapi.p",'rb').close

date_vars = ['Date']
air_df = []
for file in glob('sfOzoneByYear/*.csv'):
    air_df.append(pd.read_csv(file, parse_dates=date_vars, date_parser=pd.to_datetime))


air_df = pd.concat(air_df)
air_df.shape

In [3]:
long = air_df.iloc[3666,-1]

lat =air_df.iloc[3666,-2]

day = air_df.iloc[3666,0]


day = int(time.mktime(day.timetuple()))
url ='https://api.darksky.net/forecast/'+dskey+'/'+str(lat)+','+str(long)+','+str(day)
weather= requests.get(url)
weather.json()['daily']['data'][0]

In [4]:
day_length =[]
temp_high =[]
temp_low =[]
dew_point =[]
humidity =[]
pressure =[]
wind_speed =[]
wind_bearing =[]
cloud_cover =[]
uv_index =[]

for i in range(air_df.shape[0]):
    try:
        long = air_df.iloc[i,-1]
        lat =air_df.iloc[i,-2]
        day = air_df.iloc[i,0]
        day = pd.to_datetime(day , format ='%m/%d/%Y')
        day = int(time.mktime(day.timetuple()))
        url ='https://api.darksky.net/forecast/'+dskey+'/'+str(lat)+','+str(long)+','+str(day)
        weather= requests.get(url)
        day_length.append(weather.json()['daily']['data'][0]['sunsetTime'] - weather.json()['daily']['data'][0]['sunriseTime'])
        try:
            temp_high.append(weather.json()['daily']['data'][0]['temperatureHigh'])
        except:
            temp_high.append(weather.json()['daily']['data'][0]['temperatureMax'])
        try:
            temp_low.append(weather.json()['daily']['data'][0]['temperatureLow'])
        except:
            temp_low.append(weather.json()['daily']['data'][0]['temperatureMin'])
        dew_point.append(weather.json()['daily']['data'][0]['dewPoint'])
        humidity.append(weather.json()['daily']['data'][0]['humidity'])
        pressure.append(weather.json()['daily']['data'][0]['pressure'])
        wind_speed.append(weather.json()['daily']['data'][0]['windSpeed'])
        wind_bearing.append(weather.json()['daily']['data'][0]['windBearing'])
        cloud_cover.append(weather.json()['daily']['data'][0]['cloudCover'])
        uv_index.append(weather.json()['daily']['data'][0]['uvIndex'])
    except:
        print('broken at: ' +str(i))

    

In [5]:
air_df['day_length'] = day_length
air_df['temp_high'] = temp_high
air_df['temp_low'] = temp_low
air_df['dew_point'] = dew_point
air_df['humidity'] = humidity
air_df['pressure'] = pressure
air_df['wind_speed'] = wind_speed
air_df['wind_bearing'] = wind_bearing
air_df['cloud_cover'] = cloud_cover
air_df['uv_index'] = uv_index

In [6]:
pickle.dump(air_df, open('air_df_1.p','wb'))

In [7]:
air_df = pickle.load(open("air_df_1.p",'rb'))
open('air_df_1.p','rb').close

In [8]:
df = air_df

df.columns

df.drop(columns=['Source','Site ID','POC','UNITS', 'DAILY_AQI_VALUE',
       'Site Name', 'DAILY_OBS_COUNT', 'PERCENT_COMPLETE',
       'AQS_PARAMETER_CODE', 'AQS_PARAMETER_DESC', 'CBSA_CODE', 'CBSA_NAME',
       'STATE_CODE', 'STATE', 'COUNTY_CODE', 'COUNTY', 'SITE_LATITUDE',
       'SITE_LONGITUDE'], inplace=True)
df.set_index('Date', inplace=True)

pickle.dump(df,open('ts_with_weather_sf.p', 'wb'))

df = pickle.load(open('ts_with_weather_sf.p', 'rb'))

504